# SQLite

In [ ]:
from ipywidgets import widgets
from IPython.display import display
import time
import json
import os
import pandas as pd
import glob
from sqlalchemy import create_engine
import string
import re
import contractions
from emoji import UNICODE_EMOJI
string.punctuation
import nltk

def remove_Atsign(text):
    """Remove at sign from the input Text"""
    return re.sub(r"@\S+", "", text)

def remove_URL(text):
    """Remove URL from the input Text"""
    return re.sub(r"http\S+", "", text)

def remove_Contraction(text):
    """Remove Contraction from the input Text"""
    try:
        text = contractions.fix(text)
    except:
        pass
    return(text)

def remove_Emoji(text):
    """Remove Emoji"""
    return ''.join(c for c in text if c not in UNICODE_EMOJI['en'])

def remove_GUID(text):
    """Remove URL from the input Text"""
    return re.sub(r"[0-9a-f]{8}-[0-9a-f]{4}-[1-5][0-9a-f]{3}-[89ab][0-9a-f]{3}-[0-9a-f]{12}", "", text)

def remove_Numbers(text):
    """Remove Numbers from the input Text"""
    remove_digits = str.maketrans('', '', string.digits)
    return(text.translate(remove_digits))

def remove_Punctuation(text):
    """Remove Punctuation from the input Text"""
    """!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~"""
    text_punctuated = "".join([char for char in text if char not in string.punctuation])
    return text_punctuated

def user_id(user):
    try:
        return(user['id'])
    except:
        pass

def user_location(user):
    try:
        return(user['location'])
    except:
        pass

def extended_tweet(extended):
    try:
        return(extended['full_text'])
    except:
        pass

def user_description(user):
    try:
        return(user['description'])
    except:
        pass

def tokenize_data(text):
    """Tokenizing input text"""
    tokens = re.split('\W+', text.lower())
    return tokens

def remove_Stopwords(text):
    """Removing Stops words"""
    stopwords = nltk.corpus.stopwords.words('english')
    #Adding additional words to the list of stop words to remove them
    #addition_StopWords = []
    #stopwords.extend(addition_StopWords)
    text_stopwords = [word for word in text if word not in stopwords]
    return(text_stopwords)

def lemmatizing(text):
    """Lemmatizing the input text using WordNet and NLTK package"""
    NLTK_WNL = nltk.WordNetLemmatizer()
    text_Lem = [NLTK_WNL.lemmatize(word) for word in text]
    return(text_Lem)

def tweet_cleaning(text):
    """Cleaning and Lemmatizing Tweets"""
    text = remove_URL(text)
    text = remove_Atsign(text)
    text = remove_Emoji(text)
    text = remove_GUID(text)
    text = remove_Numbers(text)
    text = remove_Contraction(text)
    text = remove_Punctuation(text)
    text = tokenize_data(text)
    text = remove_Stopwords(text)
    text = lemmatizing(text)
    return(text)

    
datastore_Path = "D:/F-drive-31578/Twitter-2020-12/All"
tweet_Cols = ['u_id', 'u_location', 'u_description', 'id', 'extended_tweet', 'retweeted', 'geo', 'coordinates', 'place']
#tweet_Cols = ['u_id', 'u_location', 'u_description', 'id', 'text', 'retweeted', 'geo', 'coordinates', 'place']

df_List = [] #An empty list to keep DataFrames
json_Filelist = glob.glob(os.path.join(datastore_Path, '*.json'))


prog = widgets.IntProgress(continuous_update=False, min=0, max=len(json_Filelist), 
                            description=  '0/' + str(len(json_Filelist)) + '  ', orientation='horizontal',
                            style={'bar_style': 'success'})# instantiate the bar
display(prog)# display the bar

#db = sqlite3.connect("F:/Tweets/Tweets.sqlite")
engine = create_engine('sqlite:///C:\Tweets\\Tweets_clean.db', echo = False)
sqlite_connection = engine.connect()

for file in json_Filelist:
    df_Temp = pd.read_json(file, lines=True)
    
    df_Temp = df_Temp[df_Temp['lang'] == 'en']
    df_Temp['u_id'] = df_Temp['user'].apply(lambda x: user_id(x))
    
    df_Temp['u_location'] = df_Temp['user'].apply(lambda x: user_location(x))
    df_Temp = df_Temp[df_Temp['u_location'].notnull()]

    df_Temp = df_Temp[df_Temp['extended_tweet'].notnull()]
    df_Temp['extended_tweet'] = df_Temp['extended_tweet'].apply(lambda x: extended_tweet(x))

    df_Temp['u_description'] = df_Temp['user'].apply(lambda x: user_description(x))
    df_Temp['u_description'] = df_Temp['u_description'].apply(lambda x: tweet_cleaning(x))
    
    df_Temp['extended_tweet'] = df_Temp['extended_tweet'].apply(lambda x: tweet_cleaning(x))
    
    #df_Temp = df_Temp[df_Temp['extended_tweet'].str.len() >= 50]
    df_Temp[tweet_Cols].astype(str).to_sql("Tweets", sqlite_connection, if_exists="append")
    prog.value += 1 # signal to increment the progress bar
    prog.description =  str(prog.value) + '/' + str(len(json_Filelist)) + '  '

sqlite_connection.close()

In [78]:
sqlite_connection.close()

In [42]:
sqlite_connection = engine.connect()
df11 = pd.read_sql('SELECT * FROM Tweets', sqlite_connection)
df11.head()

,index,u_id,u_location,u_description,id,text,extended_tweet,retweeted,geo,coordinates,place
0,22,3232688427,United Kingdom,"20 | Queer | He/They | I love RGG, DMC, MGS, S...",1.346380920534786e+18,@JudgementKinsey Garfields third bomb: Bites t...,nan,0.0,nan,nan,nan
1,40,3099677072,"Islamabad, Pakistan",‏‏‏‏‏‏آتنگوادی!,1.346380920526426e+18,RT @loishh: the art ...,nan,0.0,nan,nan,nan
2,51,4755630367,"2 Leman Street, London E1 8FA",Insurance Business UK is a website with multip...,1.3463809205347942e+18,#Insurance companies have turned to #artificia...,"{'full_text': ""#Insurance companies have turne...",0.0,nan,nan,nan
3,53,14371794,"Crewe, Cheshire","Singer/songwriter, promoter, environmentalist....",1.3463809205389926e+18,On the twelfth day of Christmas is a cover of ...,"{'full_text': ""On the twelfth day of Christmas...",0.0,nan,nan,nan
4,54,709368941592846336,"Cambridgeshire, England",Healthy You in Cambridgeshire & Peterborough i...,1.3463809205348024e+18,As a lot of us are making #NewYearResolutions ...,{'full_text': 'As a lot of us are making #NewY...,0.0,nan,nan,nan


# PKL Combiner

In [11]:
list_dfs = []
pkl_Path = "F:/Tweets"
pkl_Filelist = glob.glob(os.path.join(pkl_Path, '*.pkl'))
prog = widgets.IntProgress(continuous_update=False, min=0, max=len(pkl_Filelist), 
                            description=  '0 / ' + str(len(pkl_Filelist)) + '  ', orientation='horizontal',
                            style={'bar_style': 'success'})# instantiate the bar
display(prog)# display the bar
#counter = 0
for file in pkl_Filelist:
    try:
        list_dfs.append(pd.read_pickle(file))
        #counter += 1
        prog.value += 1 # signal to increment the progress bar
        prog.description =  str(prog.value) + ' / ' + str(len(pkl_Filelist)) + '  '
    except:
        pass


combined_df = pd.concat(list_dfs)
combined_df.to_pickle('F:/Tweets/combined.pkl')
combined_df.head()

IntProgress(value=0, description='0 / 3  ', max=3)

,id,text,extended_tweet,u_id,u_location,u_description,retweeted
9,1.333697e+18,Y am I getting photoshoot and brand ambassador...,NaN,1017370991301832704,♡•,—yolo sö chile,0.0
42,1.333697e+18,"Have a good day, everyone (except you, Laurenc...",NaN,77269789,"Co. Durham, England","Writer with bylines @ Digital Spy, SYFY, Scree...",0.0
50,1.333697e+18,@lexsion @catturd2 @POTUS You need a better ho...,NaN,786704913447395328,United States,"Ex-Democrat 18 years. Part Republican, part Li...",0.0
51,1.333697e+18,Each Tuesday in December we want to celebrate ...,{'full_text': 'Each Tuesday in December we wan...,4869678789,"Dublin City, Ireland",SHARECITY is a @ERC_Research funded research p...,0.0
52,1.333697e+18,📢 CALL FOR PARTICIPANTS | NYC Youth Mental Wel...,{'full_text': '📢 CALL FOR PARTICIPANTS | NYC Y...,178367902,Republic of the Philippines,The Voice and Advocate of the Youth,0.0


# Multiple finder

In [13]:
import pandas as pd

# duplicates finder
def getDuplicateUsers(df):
	duplicateUsers = set()
	for x in range(df.shape[1]):
		usr = df.iloc[x]['u_id']
		for y in range(x + 1, df.shape[1]):
			otherUsr = df.iloc[y]['u_id']
			if usr == otherUsr:
				duplicateUsers.add(usr)
	return list(duplicateUsers)

df_Tweets = pd.read_pickle('F:/Tweets/combined.pkl')
duplicateColNames = getDuplicateUsers(df_Tweets)
print('Duplicate Columns are :', duplicateColNames)


Duplicate Columns are : []


In [19]:
db = sqlite3.connect("F:/Tweets/Tweets.sqlite")
db.close()